In [1020]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1004]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
sys.path.append("/space/ge38huj/state_approximation/")

In [1005]:
from misc import mps_overlap, mps_entanglement_spectrum, mpo_on_mpo
from state_approximation import mps2mpo, mpo2mps, diagonal_expansion, multiple_diagonal_expansions,\
    entanglement_entropy
from disentanglers import disentangle_S2, renyi_entropy
from rfunc import pad_mps, pad
import pickle
from tebd import tebd
from contraction_shifted import contract_diagonal_expansion_top, contract_diagonal_expansion_bottom,\
                                contract_mpo, contract_diagonal_expansion_full, contract_series_diagonal_expansion
from moses_variational_shifted import optimize_single_site, _optimize_single_site_sweep_fast, apply_Us_to_A,\
                                      optimize_single_site_sweep_fast
from random_mps import *
from quantum_circuits import *

In [379]:
def rotation(phi):
    return np.array([[np.cos(phi), -np.sin(phi)],[np.sin(phi), np.cos(phi)]])

In [995]:
Psi = [np.random.rand(2,1,1) for i in range(10)]
norm = mps_overlap(Psi, Psi)
Psi[0] /= np.sqrt(norm)

In [1003]:
np.array(np.linspace(32, 1, 6), dtype=np.int)

array([32, 25, 19, 13,  7,  1])

* S should drop by log2 at each step
* to see how entanglement is removed
* look at truncation error at each step
* check if using the renyi 1/2 disentangler does a better job
* circuit of depth 1, is var moses equivalent 

In [988]:
state, _, _ = tebd(L=10, g=1.5, dt=0.1)
with open("/space/ge38huj/state_approximation/sh_data/T0.1.pkl", "rb") as f:
    sh_state = pickle.load(f)
    
Psi = mps2mpo(sh_state.copy())
#Psi = bell_pair_mps(10, 9)
#Psi = mps2mpo(Psi)
#As, Lambda, info = multiple_diagonal_expansions(Psi,1, mode='exact', num_sweeps=10)

print("Finished")

Finished


In [992]:
A, Lambda = diagonal_expansion(Psi, eta=1, num_sweeps=100)
#out = contract_series_diagonal_expansion(As, Lambda, mode='top')
out = contract_diagonal_expansion_top(A, Lambda)
np.linalg.norm(mps_overlap(Psi, out))

0.9999999998184644

In [793]:
entanglement_entropy(Lambda)

0.5224537526860992

In [736]:
out = contract_series_diagonal_expansion(As, Lambda, mode='top')
np.linalg.norm(mps_overlap(out, Psi))

0.9652182212015843

In [1006]:
with open("/space/ge38huj/state_approximation/sh_data_chi32/T2.5.pkl", "rb") as f:
    Psi = pickle.load(f)

In [1007]:
with open("/space/ge38huj/state_approximation/mm_initial_guesses_chi32/3_layers_T2.5.pkl", "rb") as f:
    Ulist = pickle.load(f)

In [1017]:
Ulist = get_initial_guess(Psi, 5)

Completed initial contraction, starting optimizations
0
0.0002745510557382974
1
0.08731913326666044
2
0.3493149139535827
3
0.4720336436736801
4
0.4907949167349893
5
0.47975465459223693
6
0.4666576268834989
7
0.45989696250774437
8
0.45734284558409377
9
0.4564958036513166
10
0.45623160000186325
11
0.4561544778188904


In [1018]:
out = generate_state_from_unitary_list(Ulist)

Starting from trivial state


In [1019]:
1-np.linalg.norm(mps_overlap(Psi, out))**2

0.792028104982906

In [1023]:
with open("/space/ge38huj/state_approximation/mm_initial_guesses_long_chi32/5_layers_exponential_schedule_T4.0.pkl", "rb") as f:
    ulist = pickle.load(f)